In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1DvX5XWC8mluYULlxP19dfwrDdQIYawHu", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/00_intro.mp3"))

In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# Action Tokenization & Behavioral Cloning: Teaching a Language Model to Drive by Imitation

**Notebook 2 — VLAs for Autonomous Driving Series | Vizuara**

**Estimated time: ~60 minutes**

Let us start with a simple question. Language models output words — sequences of discrete tokens like "the", "cat", "sat". But a car needs continuous signals: steer 15.3 degrees left, accelerate at 2.1 m/s squared, follow a smooth curve through the intersection. How do we bridge this gap? How do we get a model that "speaks" in English to also "speak" in steering angles?

The answer is beautifully simple: we turn driving actions into words. We discretize continuous steering angles into bins and add them to the model's vocabulary as new tokens. Then we train the model to imitate expert drivers, predicting the correct action token at each timestep — exactly the way it would predict the next word in a sentence.

By the end of this notebook, you will build a complete action tokenization system, train a behavioral cloning model that learns to imitate expert driving trajectories, and see it in action across five different driving maneuvers.

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://course-creator-brown.vercel.app/courses/vla-autonomous-driving/practice/2/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 1 — Why Does This Matter?

Here is the core tension. Systems like RT-2 from Google DeepMind and EMMA from Waymo are built on large language models — models with a fixed vocabulary of text tokens. These models are extraordinarily good at understanding images, following instructions, and reasoning about situations. But they were never designed to output continuous numbers like "steer 14.7 degrees."

So researchers asked: **what if we just made steering angles part of the vocabulary?**

This is action tokenization. Instead of outputting the continuous value 15.0 degrees, the model outputs **token 170** — a discrete symbol that represents the bin containing 15.0 degrees. The car's controller then converts token 170 back to approximately 15.0 degrees and turns the wheel.

This approach is powerful because it lets us use the **exact same training recipe** as language modeling. The model predicts the next token in a sequence — sometimes that token is a word describing the scene, and sometimes it is an action token telling the car what to do. Same architecture, same loss function, same training loop. This is exactly what we want.

But the question remains: does discretizing continuous actions lose too much precision? And how do we actually train a model to imitate expert drivers? Let us find out.

In [ ]:
#@title 🎧 Listen: Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 2 — Building Intuition (No Code)

### The Sheet Music Analogy

Imagine you are listening to a guitar solo — a continuous, flowing melody with subtle bends, vibrato, and slides between notes. Now imagine writing that solo as sheet music. Sheet music uses discrete symbols: quarter notes, half notes, specific pitches on a staff. You lose some of the continuous nuance — the exact micro-bend between two frets, the precise vibrato speed. But if your notation system is detailed enough (32nd notes, bend markers, dynamics), you can recreate the solo with remarkable fidelity.

Action tokenization does the same thing for driving. The continuous "melody" of steering is written as discrete "notes" — bin indices — that a language model can read and produce. With enough bins (256 is typical), the notation is precise enough that the reconstruction error is imperceptible to the car's physics.

### The Thermometer Analogy

Think of a thermometer. The actual temperature outside is continuous — it might be exactly 72.347 degrees Fahrenheit. But your weather app shows "72 degrees." It rounds to the nearest integer. With 256 bins spanning a 90-degree steering range, we are rounding to the nearest 0.35 degrees. That is less than the mechanical play in most steering systems. The car cannot even tell the difference.

### How It Fits Into the VLA Pipeline

In a Vision-Language-Action model for driving:
1. A camera image goes in.
2. A language description of the driving task goes in (e.g., "follow the road, turn left at the intersection").
3. The model outputs a sequence of action tokens — one per timestep.
4. Each action token is detokenized back to a continuous steering angle.
5. The car executes the steering command.

The training data consists of triplets: (camera image, language description, expert action). We train the model to predict the expert's action token given the image and language — this is **behavioral cloning**.

### Think About This

If we use 256 bins for a steering range of [-45 degrees, 45 degrees], what is the maximum tokenization error? The total range is 90 degrees, divided into 255 intervals (256 bins means 255 gaps between them). That gives us 90 / 255 = 0.353 degrees per bin. The maximum error from rounding is half a bin width: **0.176 degrees**. That is remarkably precise — far below what a human driver can consistently achieve.

In [ ]:
#@title 🎧 Listen: Mathematics
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_mathematics.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 3 — The Mathematics

Let us now formalize these ideas. We will work through three equations, and for each one, we will plug in concrete numbers so you can trace through the computation yourself.

### Equation 1: Action Tokenization

Given a continuous action $a$ (say, a steering angle), we convert it to a discrete token index:

$$a_{\text{token}} = \text{round}\left(\frac{a - a_{\min}}{a_{\max} - a_{\min}} \times (N_{\text{bins}} - 1)\right)$$

Let us break this down computationally, step by step:

1. **Subtract $a_{\min}$**: shifts the range so it starts at zero.
2. **Divide by $(a_{\max} - a_{\min})$**: normalizes to $[0, 1]$.
3. **Multiply by $(N_{\text{bins}} - 1)$**: scales to $[0, N_{\text{bins}} - 1]$.
4. **Round**: snaps to the nearest integer. That integer IS the token.

**Worked example:** A steering angle of 15 degrees, with range $[-45, 45]$ and 256 bins.

$$a_{\text{token}} = \text{round}\left(\frac{15 - (-45)}{45 - (-45)} \times (256 - 1)\right) = \text{round}\left(\frac{60}{90} \times 255\right) = \text{round}(170.0) = 170$$

So steering 15 degrees to the right maps to **token 170**. This is exactly what we want.

### Equation 2: Detokenization (The Inverse)

To convert a token back to a continuous action:

$$a = \frac{a_{\text{token}}}{N_{\text{bins}} - 1} \times (a_{\max} - a_{\min}) + a_{\min}$$

**Worked example:** Token 170 back to a steering angle.

$$a = \frac{170}{255} \times (45 - (-45)) + (-45) = 0.6667 \times 90 + (-45) = 60.0 - 45.0 = 15.0°$$

Perfect roundtrip. Token 170 reconstructs exactly to 15.0 degrees. Not all values will reconstruct perfectly — values that fall between bin centers will have a small rounding error — but with 256 bins, the maximum error is only 0.176 degrees.

### Equation 3: Behavioral Cloning Loss

Now, how do we train a model to imitate expert drivers? At each timestep $t$, the model sees an image $I_t$ and language instruction $\ell$, and predicts a probability distribution over all $N_{\text{bins}}$ action tokens. The behavioral cloning loss is:

$$\mathcal{L} = -\sum_{t=1}^{T} \log p_\theta(a_t \mid I_t, \ell)$$

This is cross-entropy loss. For each timestep, the model outputs a distribution over 256 bins. The loss measures how much probability mass the model places on the **correct** token — the one the expert chose. If the model assigns probability 1.0 to the correct token, $\log(1.0) = 0$, and the loss is zero. If it assigns probability 0.01, $\log(0.01) = -4.6$, and the loss is large. The model learns to concentrate its predictions on the expert's choices.

**Worked example:** Suppose $T = 3$ timesteps, and the model assigns these probabilities to the expert's chosen tokens:

| Timestep | Expert Token | Model's Probability | $-\log p$ |
|----------|-------------|-------------------|-----------|
| $t=1$ | 170 | 0.8 | 0.223 |
| $t=2$ | 128 | 0.5 | 0.693 |
| $t=3$ | 200 | 0.9 | 0.105 |

$$\mathcal{L} = 0.223 + 0.693 + 0.105 = 1.021$$

If the model improved and assigned probability 0.95 to all three expert tokens, the loss would drop to $3 \times 0.051 = 0.154$. Lower loss means the model is more confident about the expert's actions. This is exactly what we want.

In [ ]:
#@title 🎧 Listen: Setup And Tokenizer
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_setup_and_tokenizer.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 4 — Let Us Build It, Component by Component

Now let us implement everything we just discussed. We will build four components: the tokenizer, a synthetic driving environment, a training dataset generator, and a behavioral cloning model.

### 4.0 Setup

In [ ]:
# Setup — run this cell first
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
import matplotlib.animation as animation
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

### 4.1 Action Tokenizer

This is the bridge between the continuous world of steering angles and the discrete world of language model tokens. Let us implement the exact formulas from Section 3.

In [ ]:
class ActionTokenizer:
    """
    Convert continuous actions (steering angles) to discrete tokens and back.

    This is exactly how RT-2 and EMMA handle action outputs —
    they discretize continuous actions into bins and treat them as vocabulary tokens.
    """
    def __init__(self, action_min: float, action_max: float, n_bins: int = 256):
        self.action_min = action_min
        self.action_max = action_max
        self.n_bins = n_bins
        self.bin_width = (action_max - action_min) / (n_bins - 1)

    def tokenize(self, action: float) -> int:
        """Continuous action --> discrete token index."""
        # Step 1-2: Normalize to [0, 1]
        normalized = (action - self.action_min) / (self.action_max - self.action_min)
        # Step 3: Scale to [0, n_bins - 1]
        scaled = normalized * (self.n_bins - 1)
        # Step 4: Round and clamp
        token = int(round(scaled))
        return max(0, min(self.n_bins - 1, token))

    def detokenize(self, token: int) -> float:
        """Discrete token index --> continuous action."""
        normalized = token / (self.n_bins - 1)
        return normalized * (self.action_max - self.action_min) + self.action_min

    def tokenize_batch(self, actions: torch.Tensor) -> torch.Tensor:
        """Batch tokenization for training tensors."""
        normalized = (actions - self.action_min) / (self.action_max - self.action_min)
        tokens = torch.round(normalized * (self.n_bins - 1)).long()
        return tokens.clamp(0, self.n_bins - 1)

    def detokenize_batch(self, tokens: torch.Tensor) -> torch.Tensor:
        """Batch detokenization for inference tensors."""
        normalized = tokens.float() / (self.n_bins - 1)
        return normalized * (self.action_max - self.action_min) + self.action_min

# --- Verify with our worked example ---
tokenizer_demo = ActionTokenizer(action_min=-45.0, action_max=45.0, n_bins=256)

# Forward: 15 degrees --> token
token = tokenizer_demo.tokenize(15.0)
print(f"Steering angle 15.0° --> Token {token}")

# Inverse: token --> degrees
recovered = tokenizer_demo.detokenize(token)
print(f"Token {token} --> Steering angle {recovered:.1f}°")

# Edge cases
print(f"\nEdge cases:")
print(f"  -45.0° --> Token {tokenizer_demo.tokenize(-45.0)}")
print(f"   0.0°  --> Token {tokenizer_demo.tokenize(0.0)}")
print(f"  45.0°  --> Token {tokenizer_demo.tokenize(45.0)}")
print(f"\nBin width: {tokenizer_demo.bin_width:.3f}°")
print(f"Max roundtrip error: {tokenizer_demo.bin_width / 2:.3f}°")

In [ ]:
#@title 🎧 Listen: Tokenizer Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_tokenizer_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Now let us visualize how precise this tokenization actually is. We will compute the roundtrip error for every angle in the range, and also show how the number of bins affects precision.

In [ ]:
# Visualize tokenization precision
tokenizer_viz = ActionTokenizer(action_min=-45.0, action_max=45.0, n_bins=256)

angles = np.linspace(-45, 45, 1000)
roundtrip_errors = []
for a in angles:
    token = tokenizer_viz.tokenize(a)
    recovered = tokenizer_viz.detokenize(token)
    roundtrip_errors.append(abs(a - recovered))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

# Left: roundtrip error across the range
ax1.plot(angles, roundtrip_errors, 'steelblue', alpha=0.7, linewidth=1)
ax1.axhline(y=tokenizer_viz.bin_width / 2, color='crimson', linestyle='--',
            linewidth=2, label=f'Max error: {tokenizer_viz.bin_width / 2:.3f}°')
ax1.set_xlabel('Original Steering Angle (°)', fontsize=12)
ax1.set_ylabel('Roundtrip Error (°)', fontsize=12)
ax1.set_title('Tokenization Error Across Full Range', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Right: precision vs number of bins
bin_counts = [8, 16, 32, 64, 128, 256, 512, 1024]
max_errors = [90.0 / (2 * (b - 1)) for b in bin_counts]
colors = ['#d32f2f' if e > 1.0 else '#f57c00' if e > 0.5 else '#388e3c' for e in max_errors]
bars = ax2.bar(range(len(bin_counts)), max_errors, color=colors, edgecolor='white', linewidth=0.5)
ax2.set_xticks(range(len(bin_counts)))
ax2.set_xticklabels(bin_counts)
ax2.set_xlabel('Number of Bins', fontsize=12)
ax2.set_ylabel('Max Error (°)', fontsize=12)
ax2.set_title('Precision vs. Number of Bins', fontsize=13, fontweight='bold')
ax2.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5, label='0.5° threshold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, val in zip(bars, max_errors):
    ax2.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.1,
             f'{val:.2f}°', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("With 256 bins, the max steering error is only 0.176° — barely noticeable!")
print("Even 64 bins gives sub-degree precision. This is why tokenization works so well.")

In [ ]:
#@title 🎧 Listen: Driving Env
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/06_driving_env.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.2 Synthetic Driving Environment

We need expert driving data to train our behavioral cloning model. In a real system like EMMA, this comes from thousands of hours of human driving. For our notebook, we will create a simple 2D driving environment with five trajectory types: straight driving, lane changes, left turns, right turns, and S-curves.

In [ ]:
class SimpleDrivingEnv:
    """
    A simple 2D driving environment that generates expert trajectories.

    In a real VLA system, expert data comes from human drivers.
    Here we use analytical curves to simulate expert behavior.
    """
    def __init__(self, n_waypoints: int = 50):
        self.n_waypoints = n_waypoints

    def generate_expert_trajectory(self, trajectory_type: str = 'lane_change') -> np.ndarray:
        """Generate an expert trajectory as (n_waypoints, 2) array of [x, y] positions."""
        t = np.linspace(0, 1, self.n_waypoints)

        if trajectory_type == 'straight':
            x = t * 50
            y = np.zeros_like(t)
        elif trajectory_type == 'lane_change':
            x = t * 50
            y = 3.5 * (1 / (1 + np.exp(-10 * (t - 0.5))))  # Smooth sigmoid
        elif trajectory_type == 'left_turn':
            theta = t * np.pi / 2
            r = 20
            x = r * np.sin(theta)
            y = r * (1 - np.cos(theta))
        elif trajectory_type == 'right_turn':
            theta = t * np.pi / 2
            r = 20
            x = r * np.sin(theta)
            y = -r * (1 - np.cos(theta))
        elif trajectory_type == 's_curve':
            x = t * 50
            y = 3.0 * np.sin(2 * np.pi * t)
        else:
            raise ValueError(f"Unknown trajectory type: {trajectory_type}")

        # Add small noise for realistic variety
        x += np.random.normal(0, 0.1, len(t))
        y += np.random.normal(0, 0.1, len(t))

        return np.stack([x, y], axis=1)  # (n_waypoints, 2)

    def trajectory_to_actions(self, trajectory: np.ndarray) -> np.ndarray:
        """Convert waypoint trajectory to steering angles between consecutive points."""
        dx = np.diff(trajectory[:, 0])
        dy = np.diff(trajectory[:, 1])
        angles = np.degrees(np.arctan2(dy, dx))
        return angles  # (n_waypoints - 1,)

env = SimpleDrivingEnv(n_waypoints=50)
print(f"Environment created with {env.n_waypoints} waypoints per trajectory.")
print(f"Each trajectory produces {env.n_waypoints - 1} steering actions.")

In [ ]:
#@title 🎧 Listen: Trajectory Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/07_trajectory_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Let us visualize the five trajectory types. Each one represents a different driving maneuver that our behavioral cloning model will need to learn.

In [ ]:
# Visualize all five expert trajectory types
fig, axes = plt.subplots(1, 5, figsize=(22, 4))
trajectory_types = ['straight', 'lane_change', 'left_turn', 'right_turn', 's_curve']
colors = ['#1976d2', '#388e3c', '#f57c00', '#d32f2f', '#7b1fa2']

np.random.seed(42)  # Consistent visualization
for ax, ttype, color in zip(axes, trajectory_types, colors):
    # Plot 3 samples to show variance
    for sample in range(3):
        traj = env.generate_expert_trajectory(ttype)
        alpha = 1.0 if sample == 0 else 0.3
        lw = 2.5 if sample == 0 else 1.0
        ax.plot(traj[:, 0], traj[:, 1], color=color, linewidth=lw, alpha=alpha)
        if sample == 0:
            ax.plot(traj[0, 0], traj[0, 1], 'o', color='green', markersize=8, zorder=5)
            ax.plot(traj[-1, 0], traj[-1, 1], 's', color='red', markersize=8, zorder=5)

    ax.set_title(ttype.replace('_', ' ').title(), fontsize=13, fontweight='bold')
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')

# Add a legend manually
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='green', linestyle='None', markersize=8, label='Start'),
    Line2D([0], [0], marker='s', color='red', linestyle='None', markersize=8, label='End'),
    Line2D([0], [0], color='gray', linewidth=1, alpha=0.4, label='Variations (noise)')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3, fontsize=11,
           bbox_to_anchor=(0.5, -0.05))

plt.suptitle("Expert Driving Trajectories — Our Training Data", fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("Each trajectory type represents a different driving maneuver.")
print("The noise across samples simulates natural driving variation.")

In [ ]:
#@title 🎧 Listen: Dataset
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/08_dataset.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.3 Generating the Training Dataset

Now we need to create training data in the format our model expects: pairs of (context features, tokenized action sequences). In a real VLA, the context features would come from a vision encoder processing camera images. Here, we use a simplified representation: a one-hot encoding of the trajectory type plus the first few waypoints as spatial context.

In [ ]:
def generate_dataset(env, tokenizer, n_samples=5000):
    """
    Generate a training dataset of (context_features, action_token_sequences).

    Each sample consists of:
      - features: one-hot trajectory type (5) + first 5 waypoints flattened (10) = 15 dims
      - tokens: tokenized steering angles for the full trajectory (49 timesteps)
    """
    trajectory_types = ['straight', 'lane_change', 'left_turn', 'right_turn', 's_curve']
    type_to_idx = {t: i for i, t in enumerate(trajectory_types)}

    all_features = []
    all_tokens = []

    for _ in range(n_samples):
        # Pick a random maneuver type
        ttype = np.random.choice(trajectory_types)
        traj = env.generate_expert_trajectory(ttype)
        actions = env.trajectory_to_actions(traj)

        # Build context features
        type_onehot = np.zeros(5)
        type_onehot[type_to_idx[ttype]] = 1.0
        context = traj[:5].flatten()  # First 5 waypoints as observation
        features = np.concatenate([type_onehot, context])  # 15-dim feature vector

        # Tokenize the expert's steering actions
        action_tokens = [tokenizer.tokenize(a) for a in actions]

        all_features.append(features)
        all_tokens.append(action_tokens)

    features_tensor = torch.FloatTensor(np.array(all_features))
    tokens_tensor = torch.LongTensor(np.array(all_tokens))

    return features_tensor, tokens_tensor

# Build the dataset
# Using [-90, 90] range to accommodate all possible steering angles from our trajectories
tokenizer = ActionTokenizer(action_min=-90.0, action_max=90.0, n_bins=256)

np.random.seed(SEED)
features, tokens = generate_dataset(env, tokenizer, n_samples=5000)

# Train/test split
N_TRAIN = 4000
train_features, test_features = features[:N_TRAIN], features[N_TRAIN:]
train_tokens, test_tokens = tokens[:N_TRAIN], tokens[N_TRAIN:]

print(f"Dataset generated:")
print(f"  Training samples:       {N_TRAIN}")
print(f"  Test samples:           {len(test_features)}")
print(f"  Feature dimensionality: {features.shape[1]}")
print(f"  Action sequence length: {tokens.shape[1]} timesteps")
print(f"  Token vocabulary size:  {tokenizer.n_bins} bins")
print(f"\nToken distribution — min: {tokens.min().item()}, max: {tokens.max().item()}, "
      f"mean: {tokens.float().mean().item():.1f}")

In [ ]:
#@title 🎧 Listen: Model
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/09_model.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.4 The Behavioral Cloning Model

Now for the model itself. Our behavioral cloning model takes context features as input and predicts a probability distribution over 256 bins for each of the 49 action timesteps. This is exactly analogous to how a language model predicts a probability distribution over vocabulary tokens for each position in a sentence.

The key architectural choice: we use separate linear heads for each timestep. In a real VLA like EMMA, this would be an autoregressive transformer that predicts one action token at a time, conditioned on all previous tokens. Our simplified version predicts all timesteps in parallel, but the training objective — cross-entropy over discrete bins — is identical.

In [ ]:
class BehavioralCloningModel(nn.Module):
    """
    A model that learns to predict expert action tokens from context features.

    Architecture:
      - Encoder: 2-layer MLP that processes context features
      - Action heads: one linear layer per timestep, predicting a distribution over bins

    In a real VLA, the encoder would be a vision-language model (e.g., PaLM-E),
    and the action heads would be an autoregressive transformer decoder.
    """
    def __init__(self, input_dim: int, n_actions: int, n_bins: int = 256, hidden_dim: int = 256):
        super().__init__()
        self.n_actions = n_actions
        self.n_bins = n_bins

        # Shared feature encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
        )

        # One classification head per action timestep
        # Each head predicts a distribution over n_bins
        self.action_heads = nn.ModuleList([
            nn.Linear(hidden_dim, n_bins) for _ in range(n_actions)
        ])

    def forward(self, features: torch.Tensor) -> torch.Tensor:
        """
        Args:
            features: (batch_size, input_dim)
        Returns:
            logits: (batch_size, n_actions, n_bins) — unnormalized log-probabilities
        """
        h = self.encoder(features)
        logits = torch.stack([head(h) for head in self.action_heads], dim=1)
        return logits

# Create the model
N_ACTIONS = 49  # n_waypoints - 1
model = BehavioralCloningModel(
    input_dim=15,
    n_actions=N_ACTIONS,
    n_bins=256,
    hidden_dim=256
).to(DEVICE)

n_params = sum(p.numel() for p in model.parameters())
print(f"Model created with {n_params:,} parameters")
print(f"  Input:  15-dimensional context features")
print(f"  Output: {N_ACTIONS} action distributions, each over {256} bins")
print(f"  This is equivalent to predicting {N_ACTIONS} 'words' from a vocabulary of {256}.")

In [ ]:
#@title 🎧 Listen: Training
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/10_training.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.5 Training Loop — Behavioral Cloning

Here is where the behavioral cloning loss from Section 3 comes to life. For every batch, we compute the cross-entropy between the model's predicted distributions and the expert's chosen tokens. The model learns to assign high probability to the expert's actions.

In [ ]:
# --- Training ---
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
BATCH_SIZE = 64
N_EPOCHS = 30

train_losses = []
test_losses = []
test_accuracies = []

for epoch in range(N_EPOCHS):
    # --- Train ---
    model.train()
    epoch_loss = 0.0
    n_batches = 0
    perm = torch.randperm(N_TRAIN)

    for i in range(0, N_TRAIN, BATCH_SIZE):
        idx = perm[i:i + BATCH_SIZE]
        batch_features = train_features[idx].to(DEVICE)
        batch_tokens = train_tokens[idx].to(DEVICE)

        logits = model(batch_features)  # (batch, 49, 256)

        # Behavioral cloning loss: cross-entropy over all action positions
        loss = F.cross_entropy(
            logits.reshape(-1, 256),       # (batch * 49, 256)
            batch_tokens.reshape(-1)       # (batch * 49,)
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        n_batches += 1

    avg_train_loss = epoch_loss / n_batches
    train_losses.append(avg_train_loss)

    # --- Evaluate ---
    model.eval()
    with torch.no_grad():
        test_logits = model(test_features.to(DEVICE))
        test_loss = F.cross_entropy(
            test_logits.reshape(-1, 256),
            test_tokens.to(DEVICE).reshape(-1)
        ).item()

        # Accuracy: how often does the model predict the exact right bin?
        pred_tokens = test_logits.argmax(dim=-1)
        accuracy = (pred_tokens.cpu() == test_tokens).float().mean().item()

    test_losses.append(test_loss)
    test_accuracies.append(accuracy)

    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1:2d}/{N_EPOCHS} | "
              f"Train Loss: {avg_train_loss:.4f} | "
              f"Test Loss: {test_loss:.4f} | "
              f"Test Accuracy: {accuracy:.1%}")

print(f"\nTraining complete!")
print(f"Final test loss: {test_losses[-1]:.4f}")
print(f"Final test accuracy: {test_accuracies[-1]:.1%}")

In [ ]:
#@title 🎧 Listen: Training Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/11_training_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Let us visualize the training progress and a sample prediction. The loss should decrease steadily as the model learns to imitate the expert's steering patterns.

In [ ]:
# Training curves and a sample prediction
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss curves
axes[0].plot(train_losses, 'b-', linewidth=2, label='Train Loss')
axes[0].plot(test_losses, 'r-', linewidth=2, label='Test Loss')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Cross-Entropy Loss', fontsize=12)
axes[0].set_title('Behavioral Cloning Training', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Accuracy curve
axes[1].plot(test_accuracies, 'g-', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Exact Token Accuracy', fontsize=12)
axes[1].set_title('Test Accuracy (Exact Bin Match)', fontsize=13, fontweight='bold')
axes[1].set_ylim(0, 1)
axes[1].grid(True, alpha=0.3)

# Sample prediction: expert vs predicted steering
model.eval()
with torch.no_grad():
    sample_logits = model(test_features[:1].to(DEVICE))
    sample_pred_tokens = sample_logits.argmax(dim=-1).squeeze(0).cpu()

pred_angles = [tokenizer.detokenize(t.item()) for t in sample_pred_tokens]
true_angles = [tokenizer.detokenize(t.item()) for t in test_tokens[0]]

axes[2].plot(true_angles, 'g-', linewidth=2.5, label='Expert', alpha=0.8)
axes[2].plot(pred_angles, 'r--', linewidth=2, label='Model Prediction')
axes[2].set_xlabel('Timestep', fontsize=12)
axes[2].set_ylabel('Steering Angle (°)', fontsize=12)
axes[2].set_title('Expert vs. Predicted Steering', fontsize=13, fontweight='bold')
axes[2].legend(fontsize=11)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("The model has learned to predict steering sequences that closely match the expert.")
print("This is exactly what behavioral cloning achieves — imitation through supervised learning.")

In [ ]:
#@title 🎧 Listen: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/12_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 5 — Your Turn

Now it is time for you to implement the core ideas yourself. These exercises test the two most important concepts from this notebook: action tokenization and the behavioral cloning loss.

### TODO 1: Implement Tokenization from Scratch

Write the tokenization function without using the ActionTokenizer class. This forces you to internalize the formula.

In [ ]:
def my_tokenize(action: float, action_min: float, action_max: float, n_bins: int) -> int:
    """
    Convert a continuous action to a discrete token.

    Args:
        action:     the continuous action value (e.g., steering angle in degrees)
        action_min: minimum of the action range
        action_max: maximum of the action range
        n_bins:     number of discrete bins

    Returns:
        token: int in [0, n_bins - 1]

    Steps:
        1. Normalize action to [0, 1] range
        2. Scale to [0, n_bins - 1]
        3. Round to nearest integer
        4. Clamp to valid range
    """
    # ============ YOUR CODE HERE ============
    # Step 1: Normalize
    # normalized = ???

    # Step 2: Scale
    # scaled = ???

    # Step 3: Round
    # token = ???

    # Step 4: Clamp
    # token = ???

    # ========================================

    token = ???  # Replace with your implementation

    return token


def my_detokenize(token: int, action_min: float, action_max: float, n_bins: int) -> float:
    """
    Convert a discrete token back to a continuous action.

    Args:
        token:      the discrete token index
        action_min: minimum of the action range
        action_max: maximum of the action range
        n_bins:     number of discrete bins

    Returns:
        action: float — the reconstructed continuous value
    """
    # ============ YOUR CODE HERE ============
    # Step 1: Normalize token to [0, 1]
    # Step 2: Scale to [action_min, action_max]
    # ========================================

    action = ???  # Replace with your implementation

    return action

In [ ]:
#@title 🎧 Listen: Todo1 Verify
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/13_todo1_verify.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# Verification — all assertions must pass
assert my_tokenize(15.0, -45.0, 45.0, 256) == 170, "15° should map to token 170"
assert my_tokenize(-45.0, -45.0, 45.0, 256) == 0, "-45° should map to token 0"
assert my_tokenize(45.0, -45.0, 45.0, 256) == 255, "45° should map to token 255"
assert my_tokenize(0.0, -45.0, 45.0, 256) == 128, "0° should map to token 128 (midpoint)"

assert abs(my_detokenize(170, -45.0, 45.0, 256) - 15.0) < 0.01, "Token 170 should reconstruct to ~15°"
assert abs(my_detokenize(0, -45.0, 45.0, 256) - (-45.0)) < 0.01, "Token 0 should reconstruct to -45°"
assert abs(my_detokenize(255, -45.0, 45.0, 256) - 45.0) < 0.01, "Token 255 should reconstruct to 45°"

print("Your tokenization and detokenization are both correct!")
print("You have implemented the exact same algorithm used in RT-2 and EMMA.")

In [ ]:
#@title 🎧 Listen: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/14_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### TODO 2: Implement the Behavioral Cloning Loss

This is the loss function that drives the entire training process. Implement it from scratch using PyTorch's cross-entropy.

In [ ]:
def behavioral_cloning_loss(predicted_logits: torch.Tensor, expert_tokens: torch.Tensor) -> torch.Tensor:
    """
    Compute the behavioral cloning loss.

    This is cross-entropy over action tokens — the same loss used in language modeling,
    applied to driving actions instead of words.

    Args:
        predicted_logits: (batch_size, n_actions, n_bins) — model's raw output
        expert_tokens:    (batch_size, n_actions)         — expert's chosen tokens

    Returns:
        loss: scalar tensor — average cross-entropy loss

    Hint: F.cross_entropy expects inputs of shape (N, C) and targets of shape (N,)
          where C is the number of classes (bins). You need to reshape accordingly.
    """
    # ============ YOUR CODE HERE ============
    # Step 1: Get the shapes
    # batch_size, n_actions, n_bins = predicted_logits.shape

    # Step 2: Reshape logits to (batch_size * n_actions, n_bins)
    # flat_logits = ???

    # Step 3: Reshape expert tokens to (batch_size * n_actions,)
    # flat_tokens = ???

    # Step 4: Compute cross-entropy loss
    # loss = F.cross_entropy(flat_logits, flat_tokens)
    # ========================================

    loss = ???  # Replace with your implementation

    return loss

In [ ]:
#@title 🎧 Listen: Todo2 Verify
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/15_todo2_verify.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# Verification
torch.manual_seed(42)
test_logits_v = torch.randn(4, 10, 256)   # 4 samples, 10 actions, 256 bins
test_expert_v = torch.randint(0, 256, (4, 10))  # Expert tokens

loss_v = behavioral_cloning_loss(test_logits_v, test_expert_v)

assert loss_v.ndim == 0, f"Loss should be a scalar, got ndim={loss_v.ndim}"
assert loss_v.item() > 0, f"Loss should be positive, got {loss_v.item()}"
assert not torch.isnan(loss_v), "Loss should not be NaN"
assert abs(loss_v.item() - F.cross_entropy(
    test_logits_v.reshape(-1, 256), test_expert_v.reshape(-1)
).item()) < 1e-5, "Loss does not match expected value"

print(f"Loss computed correctly: {loss_v.item():.4f}")
print("Your implementation matches PyTorch's cross_entropy — this is the exact loss")
print("used to train behavioral cloning models like EMMA.")

In [ ]:
#@title 🎧 Listen: Final Eval
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/16_final_eval.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 6 — Final Evaluation: Expert vs. Learned Trajectories

Now let us see the full picture. We will compare the expert trajectories against our model's learned trajectories for all five driving maneuvers. This is the moment of truth — can our model, trained purely by imitating experts, actually reproduce their driving behavior?

In [ ]:
# Comprehensive comparison: expert vs. model across all 5 maneuvers
model.eval()
trajectory_types = ['straight', 'lane_change', 'left_turn', 'right_turn', 's_curve']
type_to_idx = {t: i for i, t in enumerate(trajectory_types)}

fig, axes = plt.subplots(2, 5, figsize=(24, 9))
np.random.seed(123)  # Fresh seed for evaluation trajectories

for col, ttype in enumerate(trajectory_types):
    # Generate an expert trajectory
    expert_traj = env.generate_expert_trajectory(ttype)
    expert_actions = env.trajectory_to_actions(expert_traj)

    # Build context features (same format as training)
    type_onehot = np.zeros(5)
    type_onehot[type_to_idx[ttype]] = 1.0
    context = expert_traj[:5].flatten()
    features_np = np.concatenate([type_onehot, context])
    features_t = torch.FloatTensor(features_np).unsqueeze(0).to(DEVICE)

    # Model prediction
    with torch.no_grad():
        pred_logits = model(features_t)
        pred_token_seq = pred_logits.argmax(dim=-1).squeeze(0).cpu()

    pred_actions = np.array([tokenizer.detokenize(t.item()) for t in pred_token_seq])

    # Reconstruct predicted trajectory from steering angles
    pred_traj = [expert_traj[0].copy()]
    for a in pred_actions:
        dx = np.cos(np.radians(a))
        dy = np.sin(np.radians(a))
        pred_traj.append(pred_traj[-1] + np.array([dx, dy]))
    pred_traj = np.array(pred_traj)

    # --- Top row: trajectory comparison ---
    axes[0, col].plot(expert_traj[:, 0], expert_traj[:, 1], 'g-', linewidth=2.5,
                      label='Expert', alpha=0.8)
    axes[0, col].plot(pred_traj[:, 0], pred_traj[:, 1], 'r--', linewidth=2,
                      label='Learned')
    axes[0, col].plot(expert_traj[0, 0], expert_traj[0, 1], 'ko', markersize=8, zorder=5)
    axes[0, col].set_title(ttype.replace('_', ' ').title(), fontsize=13, fontweight='bold')
    axes[0, col].set_aspect('equal')
    axes[0, col].grid(True, alpha=0.3)
    axes[0, col].legend(fontsize=9, loc='best')
    if col == 0:
        axes[0, col].set_ylabel('y position (m)', fontsize=11)

    # --- Bottom row: steering angle comparison ---
    axes[1, col].plot(expert_actions, 'g-', linewidth=2, label='Expert', alpha=0.8)
    axes[1, col].plot(pred_actions, 'r--', linewidth=1.5, label='Learned')
    axes[1, col].set_xlabel('Timestep', fontsize=11)
    axes[1, col].grid(True, alpha=0.3)
    axes[1, col].legend(fontsize=9, loc='best')
    if col == 0:
        axes[1, col].set_ylabel('Steering Angle (°)', fontsize=11)

plt.suptitle("Behavioral Cloning Results: Expert vs. Learned Driving",
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Compute overall error statistics
all_errors = []
np.random.seed(456)
for _ in range(100):
    ttype = np.random.choice(trajectory_types)
    traj = env.generate_expert_trajectory(ttype)
    actions = env.trajectory_to_actions(traj)

    type_onehot = np.zeros(5)
    type_onehot[type_to_idx[ttype]] = 1.0
    context = traj[:5].flatten()
    feat = np.concatenate([type_onehot, context])
    feat_t = torch.FloatTensor(feat).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        logits = model(feat_t)
        preds = logits.argmax(dim=-1).squeeze(0).cpu()

    pred_a = np.array([tokenizer.detokenize(t.item()) for t in preds])
    error = np.abs(pred_a - actions)
    all_errors.extend(error.tolist())

all_errors = np.array(all_errors)
print(f"\nOverall steering angle error across 100 random test trajectories:")
print(f"  Mean error:   {all_errors.mean():.2f}°")
print(f"  Median error: {np.median(all_errors):.2f}°")
print(f"  95th pctl:    {np.percentile(all_errors, 95):.2f}°")
print(f"\nThe model has learned to imitate expert driving through pure behavioral cloning.")
print("In a real system like EMMA, the same cross-entropy training objective is used —")
print("but with a vision transformer encoder processing real camera images.")

Let us also create an animated visualization showing the model's learned driving in action. This makes the behavioral cloning result tangible — you can watch the car follow the learned trajectory in real time.

In [ ]:
# Animated comparison: expert vs. learned trajectory
np.random.seed(42)
demo_traj = env.generate_expert_trajectory('s_curve')
demo_actions = env.trajectory_to_actions(demo_traj)

type_onehot = np.zeros(5)
type_onehot[type_to_idx['s_curve']] = 1.0
context = demo_traj[:5].flatten()
feat = np.concatenate([type_onehot, context])
feat_t = torch.FloatTensor(feat).unsqueeze(0).to(DEVICE)

with torch.no_grad():
    logits = model(feat_t)
    pred_tok = logits.argmax(dim=-1).squeeze(0).cpu()

pred_act = np.array([tokenizer.detokenize(t.item()) for t in pred_tok])
pred_traj_anim = [demo_traj[0].copy()]
for a in pred_act:
    dx = np.cos(np.radians(a))
    dy = np.sin(np.radians(a))
    pred_traj_anim.append(pred_traj_anim[-1] + np.array([dx, dy]))
pred_traj_anim = np.array(pred_traj_anim)

fig, ax = plt.subplots(figsize=(12, 5))

def animate(frame):
    ax.clear()
    # Full expert path (faded)
    ax.plot(demo_traj[:, 0], demo_traj[:, 1], 'g-', linewidth=1, alpha=0.3, label='Expert (full)')

    # Expert up to current frame
    end = min(frame + 1, len(demo_traj))
    ax.plot(demo_traj[:end, 0], demo_traj[:end, 1], 'g-', linewidth=2.5, label='Expert')

    # Learned up to current frame
    end_p = min(frame + 1, len(pred_traj_anim))
    ax.plot(pred_traj_anim[:end_p, 0], pred_traj_anim[:end_p, 1], 'r--', linewidth=2, label='Learned')

    # Current position markers
    if end > 0:
        ax.plot(demo_traj[end-1, 0], demo_traj[end-1, 1], 'go', markersize=12, zorder=5)
    if end_p > 0:
        ax.plot(pred_traj_anim[end_p-1, 0], pred_traj_anim[end_p-1, 1], 'rs', markersize=10, zorder=5)

    ax.set_xlim(-5, 55)
    ax.set_ylim(-6, 6)
    ax.set_xlabel('x (m)', fontsize=12)
    ax.set_ylabel('y (m)', fontsize=12)
    ax.set_title(f'S-Curve: Expert vs. Learned (t={frame})', fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=11)
    ax.grid(True, alpha=0.3)
    return []

anim = animation.FuncAnimation(fig, animate, frames=len(demo_traj), interval=100, blit=True)
plt.close(fig)
HTML(anim.to_jshtml())

In [ ]:
#@title 🎧 Listen: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/17_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 7 — Reflection and Next Steps

### What We Built

In this notebook, we implemented two fundamental building blocks used in state-of-the-art VLA driving systems:

1. **Action Tokenization** — converting continuous steering angles into discrete tokens that a language model can predict. We showed that with 256 bins, the quantization error is negligible (less than 0.18 degrees).

2. **Behavioral Cloning** — training a model to imitate expert drivers by minimizing cross-entropy loss over action tokens. The model learns to assign high probability to the expert's chosen actions, exactly like a language model learns to predict the next word.

### Reflection Questions

1. **Bin count tradeoffs:** We used 256 bins. What would happen with only 8 bins? The max error would jump to ~6.4 degrees — large enough to cause noticeable weaving. With 4096 bins? The error drops to 0.011 degrees, but the model must predict over a much larger vocabulary, which can slow convergence. 256 is the sweet spot for most driving applications.

2. **Expert quality matters:** Behavioral cloning faithfully imitates whatever the expert does — including mistakes. If the expert occasionally swerves unnecessarily, the model learns that too. This is both a strength (simple, stable training) and a weakness (garbage in, garbage out). Systems like DAgger address this by iteratively collecting corrections.

3. **Why cross-entropy and not MSE?** Cross-entropy treats action prediction as classification over bins, which handles multimodal distributions naturally. If there are two valid steering angles at a junction (slight left or slight right), cross-entropy can assign probability mass to both. MSE would average them, predicting "go straight" — which might be the worst option. This is a critical advantage for real-world driving.

### Optional Challenges

1. **Multi-dimensional tokenization:** Extend the tokenizer to handle both steering AND acceleration simultaneously. With 256 bins each, you need $256 \times 256 = 65{,}536$ joint tokens — or you can tokenize each dimension independently (just 512 tokens total). Which approach do RT-2 and EMMA use, and why?

2. **Autoregressive prediction:** Replace the parallel action heads with an LSTM or Transformer that predicts one action token at a time, conditioned on all previous predictions. This is closer to how real VLAs work. Does it improve trajectory smoothness?

3. **Covariate shift experiment:** After training, roll out the model in the environment: at each step, use the model's *own predicted* position (not the expert's) to compute the next action. Watch what happens as small errors accumulate. This is the **covariate shift** problem — the model encounters states it never saw during training because it was always shown the expert's states. How quickly do trajectories diverge?